<a href="https://colab.research.google.com/github/CuadraAlconero/IDAL_IA3_CuadraAlconero/blob/main/Keras_Generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generadores en Keras

En este cuaderno vamos a estudiar como construir un generador personalizado con la api de Keras.

Al igual que keras provee a sus usuarios con algunos generadores y capas diseñadas para el preprocesado de datos es posible construir un generador que realice operaciones definidas por los usuarios.

Para esto necesitamos utilizar la clase de Keras Sequence.

Sequence es una clase a partir de la cual se pueden diseñar otras con el objetivo de construir un generador personalizado, en este notebook vamos a construir un ejemplo de Sequence que cambia las resoluciones a la entrada antes de pasar la información a un modelo.

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /content/cats_and_dogs_filtered.zip
!unzip cats_and_dogs_filtered.zip

In [3]:
from tensorflow.keras.utils import Sequence
from PIL import Image
import os
from tensorflow import keras
import numpy as np
import pandas as pd
from random import shuffle

Comenzamos definiendo el constructor de nuestra clase.

In [23]:
class ImageResizer(Sequence):
  def __init__(self, data_folder, img_shape, batch_size=32, to_fit=True):
    self.data_folder = data_folder
    self.files = [os.path.join(path, name) for path, subdirs, files in os.walk(data_folder) for name in files]
    shuffle(self.files)
    self.batch_size = batch_size
    self.shape = img_shape
    self.to_fit = True

Tras definir el constructor necesitaremos definir los métodos \_\_getitem__ y \_\_len__.

Estos métodos son los encargados de pasarle nuevas muestras al modelo durante el entrenamiento y su implementación es obligatoria.

In [19]:
class ImageResizer(Sequence):
  def __init__(self, data_folder, img_shape, batch_size=32, to_fit=True):
    self.data_folder = data_folder
    self.files = [os.path.join(path, name) for path, subdirs, files in os.walk(data_folder) for name in files]
    shuffle(self.files)
    self.batch_size = batch_size
    self.shape = img_shape
    self.to_fit = True

  def __len__(self):
    batches = int(np.floor(len(self.files) / self.batch_size))
    return batches

  def __getitem__(self, index):
    batch_files = self.files[
                             index * self.batch_size : (index + 1) * self.batch_size
                            ]
    if self.to_fit:
      X = self._generate_X(batch_files)
      y = self._generate_Y(batch_files)
      return X, y
    else:
      X = self._generate_X(batch_files)
      return X

  def _generate_X(self, batch_files):
    shape = [self.batch_size]
    shape.extend(self.shape)
    shape.extend([3])
    X = np.zeros(shape=shape)
    for i, file_ in enumerate(batch_files):
      img = Image.open(file_)
      resize_image = img.resize(self.shape)
      array = np.array(resize_image)
      X[i, :, :, :] = array
    return X

  def _generate_Y(self, batch_files):
    y = list()
    for file_ in batch_files:
      label = file_.split("/")
      y.append(label[2])
    df_labels = pd.DataFrame(y)
    y_dummies = pd.get_dummies(df_labels).values
    return y_dummies
    

In [20]:
generator = ImageResizer("cats_and_dogs_filtered/train", (64,64), batch_size=128)

In [21]:
input_layer = keras.layers.Input(shape=(64,64,3))
conv_layer = keras.layers.Conv2D(10, (3,3), activation="relu",input_shape=(64,64,3))(input_layer)
conv_layer2 = keras.layers.Conv2D(10, (3,3), activation="relu")(conv_layer)
pooling = keras.layers.MaxPool2D(pool_size=(2,2))(conv_layer2)
flatten = keras.layers.Flatten(data_format="channels_last")(pooling)
dense = keras.layers.Dense(512, activation="tanh")(flatten)
classifier = keras.layers.Dense(2, activation="softmax")(dense)
model = keras.Model(inputs=input_layer, outputs=classifier)
model.compile(optimizer="adam", loss="categorical_crossentropy")

In [22]:
model.fit(generator, epochs=5)

Epoch 1/5
15/15 [==============================] - 15s 935ms/step - loss: 0.7393
Epoch 2/5
15/15 [==============================] - 15s 946ms/step - loss: 0.6991
Epoch 3/5
15/15 [==============================] - 14s 940ms/step - loss: 0.7005
Epoch 4/5
15/15 [==============================] - 14s 925ms/step - loss: 0.6994
Epoch 5/5
15/15 [==============================] - 14s 941ms/step - loss: 0.6963


# Ejercicio

Construíd un generador de keras e incluíd la lógica de padding descrita en el cuaderno anterior.

In [29]:
class ImageResizer(Sequence):
  def __init__(self, data_folder, img_shape, batch_size=32, to_fit=True):
    self.data_folder = data_folder
    self.files = [os.path.join(path, name) for path, subdirs, files in os.walk(data_folder) for name in files]
    shuffle(self.files)
    self.batch_size = batch_size
    self.shape = img_shape
    self.to_fit = True

  def __len__(self):
    batches = int(np.floor(len(self.files) / self.batch_size))
    return batches

  def __getitem__(self, index):
    batch_files = self.files[
                             index * self.batch_size : (index + 1) * self.batch_size
                            ]
    if self.to_fit:
      X = self._generate_X(batch_files)
      y = self._generate_Y(batch_files)
      return X, y
    else:
      X = self._generate_X(batch_files)
      return X

  def _generate_X(self, batch_files):
    shape = [self.batch_size]
    shape.extend(self.shape)
    shape.extend([3])
    X = np.zeros(shape=shape)
    for i, file_ in enumerate(batch_files):
      img = Image.open(file_)
      resize_image = self._change_resolution(img)
      padding_image = self._pad_img(resize_image)
      X[i, :, :, :] = padding_image
    return X

  def _pad_img(self, img):
    width, height = img.size
    desired_resolution = self.shape[0]
    array = np.array(img)
    if width > height:
      total_pad = desired_resolution - height
      pad = total_pad/2
      if isinstance(pad, float):
        pad_0 = int(np.ceil(pad))
        pad_1 = int(np.floor(pad))
        padded_image = np.pad(array, ((pad_0, pad_1),(0,0),(0,0)))
      else:
        padded_image = np.pad(array, ((pad, pad),(0,0),(0,0)))
    else:
      total_pad = desired_resolution - width
      pad = total_pad/2
      if isinstance(pad, float):
        pad_0 = int(np.ceil(pad))
        pad_1 = int(np.floor(pad))
        padded_image = np.pad(array, ((0,0),(pad_0, pad_1),(0,0)))
      else:
        padded_image = np.pad(array, ((0,0),(pad, pad),(0,0)))
    return padded_image

  def _change_resolution(self, img):
    desired_resolution = self.shape[0]
    width, height = img.size
    if width > height:
      aspect_relatio = width/height
      new_width = desired_resolution
      new_height = round(desired_resolution/aspect_relatio)
      resized_img = img.resize((new_width,new_height))
    else:
      aspect_relatio = height/width
      new_height = desired_resolution
      new_width = round(desired_resolution/aspect_relatio)
      resized_img = img.resize((new_width,new_height))
    return resized_img

  def _generate_Y(self, batch_files):
    y = list()
    for file_ in batch_files:
      label = file_.split("/")
      y.append(label[2])
    df_labels = pd.DataFrame(y)
    y_dummies = pd.get_dummies(df_labels).values
    return y_dummies

In [30]:
generator = ImageResizer("cats_and_dogs_filtered/train", (64,64), batch_size=128)
input_layer = keras.layers.Input(shape=(64,64,3))
conv_layer = keras.layers.Conv2D(10, (3,3), activation="relu",input_shape=(64,64,3))(input_layer)
conv_layer2 = keras.layers.Conv2D(10, (3,3), activation="relu")(conv_layer)
pooling = keras.layers.MaxPool2D(pool_size=(2,2))(conv_layer2)
flatten = keras.layers.Flatten(data_format="channels_last")(pooling)
dense = keras.layers.Dense(512, activation="tanh")(flatten)
classifier = keras.layers.Dense(2, activation="softmax")(dense)
model = keras.Model(inputs=input_layer, outputs=classifier)
model.compile(optimizer="adam", loss="categorical_crossentropy")

In [ ]:
model.fit(generator, epochs=5)

Epoch 1/5
15/15 [==============================] - 15s 964ms/step - loss: 0.8306
Epoch 2/5
 6/15 [===========>..................] - ETA: 8s - loss: 0.6980